In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import GridSearchCV
#from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv


In [3]:
def dframe(df): #read csv file
    df=pd.read_csv(df).sample(frac = 0.25).sort_values(by='Date')
    return df
df=dframe(df='../input/weather-dataset-rattle-package/weatherAUS.csv')
print(display(df.head(5)))

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
45588,2007-11-02,Canberra,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,...,80.0,36.0,1012.4,1008.4,5.0,3.0,17.5,25.7,Yes,Yes
45597,2007-11-11,Canberra,9.1,25.2,0.0,4.2,11.9,N,30.0,SE,...,74.0,34.0,1024.4,1021.1,1.0,2.0,14.6,24.0,No,No
45599,2007-11-13,Canberra,10.1,27.9,0.0,7.2,13.0,WNW,30.0,S,...,62.0,29.0,1022.0,1017.1,0.0,1.0,17.0,27.1,No,No
45605,2007-11-19,Canberra,12.4,32.3,0.6,7.4,13.0,E,39.0,NNE,...,60.0,25.0,1017.1,1013.3,1.0,3.0,20.2,31.2,No,No
45607,2007-11-21,Canberra,15.3,33.4,0.0,8.8,9.5,WNW,59.0,N,...,60.0,26.0,1012.4,1006.5,1.0,5.0,22.2,32.8,No,No


None


In [6]:
df.drop(['Date','Location','Evaporation','Sunshine'],axis=1, inplace=True)

In [10]:
X=pd.get_dummies(data=df, columns=['WindGustDir','WindDir9am','WindDir3pm','RainTomorrow'],drop_first=True).drop(['RainToday'], axis=1)

In [11]:
index,columns=X.index, X.columns
vulinNan=IterativeImputer(random_state=0).fit(X).transform(X)
X=pd.DataFrame(vulinNan, index=index,columns=columns)

/opt/conda/lib/python3.7/site-packages/sklearn/impute/_iterative.py:670: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [13]:
y = LabelEncoder().fit_transform(df['RainToday'].astype(str))

In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)

In [17]:
light=LGBMClassifier()
light.fit(X_train, y_train)

LGBMClassifier()

In [18]:
y_pred=light.predict(X_test)
accuracy_score(y_test, y_pred)

0.989825381548192

# Model Tuning

In [19]:
light.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [21]:
light_params={'learning_rate':[0.001,0.01,0.1],
            'n_estimators':[100,200,500],
            'max_depth':[3,5,8,10,12],
            'min_samples_split':[2,5,8,10]}

In [22]:
light_model=LGBMClassifier()
light_cv=GridSearchCV(light_model, light_params,cv=7, n_jobs=-1, verbose=3)

In [24]:
light_cv.fit(X_train,y_train)

Fitting 7 folds for each of 180 candidates, totalling 1260 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   23.7s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 504 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 18.2min
[Parallel(n_jobs=-1)]: Done 1144 tasks      | elapsed: 26.2min
[Parallel(n_jobs=-1)]: Done 1260 out of 1260 | elapsed: 29.5min finished


[LightGBM] [Warning] Unknown parameter: min_samples_split


GridSearchCV(cv=7, estimator=LGBMClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.01, 0.1],
                         'max_depth': [3, 5, 8, 10, 12],
                         'min_samples_split': [2, 5, 8, 10],
                         'n_estimators': [100, 200, 500]},
             verbose=3)

In [26]:
light_cv.best_params_

{'learning_rate': 0.1,
 'max_depth': 12,
 'min_samples_split': 2,
 'n_estimators': 500}

In [27]:
light_tuned=LGBMClassifier(learning_rate=0.1, max_depth=12, n_estimators=500,min_samples_split=2).fit(X_train,y_train)

In [28]:
y_pred_tuned=light_tuned.predict(X_test)
accuracy_score(y_test, y_pred_tuned)

0.9903753609239654

# CatBoost

In [41]:
from catboost import CatBoostClassifier

In [59]:
dfCat=dframe(df='../input/weather-dataset-rattle-package/weatherAUS.csv')


In [60]:
dfCat.drop(['Date','Location','Evaporation','Sunshine'],axis=1, inplace=True)

In [47]:
X=pd.get_dummies(data=dfCat, columns=['WindGustDir','WindDir9am','WindDir3pm','RainTomorrow'],drop_first=True).drop(['RainToday'], axis=1)


In [61]:
index,columns=X.index, X.columns
vulinNan=IterativeImputer(random_state=0).fit(X).transform(X)
X=pd.DataFrame(vulinNan, index=index,columns=columns)

In [70]:
y = LabelEncoder().fit_transform(dfCat['RainToday'].astype(str))

In [71]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)

In [72]:
cat=CatBoostClassifier().fit(X_train,y_train)

Learning rate set to 0.093967
0:	learn: 1.0214784	total: 21.1ms	remaining: 21.1s
1:	learn: 0.9605905	total: 38.6ms	remaining: 19.3s
2:	learn: 0.9109250	total: 55.4ms	remaining: 18.4s
3:	learn: 0.8699773	total: 73.3ms	remaining: 18.2s
4:	learn: 0.8357683	total: 89.6ms	remaining: 17.8s
5:	learn: 0.8070681	total: 106ms	remaining: 17.6s
6:	learn: 0.7827105	total: 123ms	remaining: 17.4s
7:	learn: 0.7617404	total: 139ms	remaining: 17.2s
8:	learn: 0.7437180	total: 147ms	remaining: 16.2s
9:	learn: 0.7282562	total: 164ms	remaining: 16.2s
10:	learn: 0.7148887	total: 181ms	remaining: 16.2s
11:	learn: 0.7032077	total: 199ms	remaining: 16.3s
12:	learn: 0.6930230	total: 216ms	remaining: 16.4s
13:	learn: 0.6841617	total: 232ms	remaining: 16.4s
14:	learn: 0.6763469	total: 248ms	remaining: 16.3s
15:	learn: 0.6695901	total: 265ms	remaining: 16.3s
16:	learn: 0.6636455	total: 283ms	remaining: 16.4s
17:	learn: 0.6584395	total: 299ms	remaining: 16.3s
18:	learn: 0.6538398	total: 316ms	remaining: 16.3s
19:	le

In [73]:
y_pred=cat.predict(X_test)
accuracy_score(y_test,y_pred)

0.7585590540354736